# Players dataframes with t-1 variables

En este script nos dedicaremos a crear una base de datos limpia segmentada por hitters y pitchers. Debido a que es totalmente análogo al script para variables del mismo periodo $t$, se omitirán solo las explicaciones del código reutilizado.

- **Visualización del contenido de las bases de datos.**
- **Limpieza de la base de datos y exportación.**
- **Creación de indicador de si el jugador es agente libre.**

Importemos los modulos necesarios así como especificar la configuración deseada.

In [1]:
import pandas as pd
import numpy as np
import math
import os
import warnings
import statsmodels.api as sm
from matplotlib.colors import ListedColormap
from termcolor import colored
print('Modulos importados')

Modulos importados


In [2]:
# Configuraciones
warnings.filterwarnings('ignore')

In [3]:
# Directorio de trabajo
print("Directorio de trabajo previo: " + str(os.getcwd()))
# Cambiemoslo
os.chdir('/home/usuario/Documentos/Github/Proyectos/MLB_HN/')

Directorio de trabajo previo: /home/usuario/Documentos/Github/Proyectos/MLB_HN/ETL_Scripts/Whole_Contract


In [4]:
# Veamos el directorio actual de trabajo
print(os.getcwd())
# El directorio anterior es el correcto, pero si no lo fuese, hacemos lo sigueinte:
path = '/home/usuario/Documentos/Github/Proyectos/MLB_HN'
print("Nuevo directorio de trabajo: " + str(os.chdir(path)))

/home/usuario/Documentos/Github/Proyectos/MLB_HN
Nuevo directorio de trabajo: None


## Visualización de las bases de datos

### Equipos por estado

In [5]:
states = 'Data/Teams/team_states.csv'
df_states = pd.read_csv(states)

In [6]:
df_states.head()

,Estado,Cantidad de equipos
0,Alabama,0
1,Alaska,0
2,Arizona,1
3,Arkansas,0
4,California,5


### Acrónimos

Nos servirá como llave intermedia para unificar las bases de datos de los equipos

In [7]:
acronym = 'Data/Teams/team_acronym.csv'
df_acronym = pd.read_csv(acronym)

In [8]:
df_acronym.head()

,Equipo,Acronimo,Estado
0,Arizona Diamondbacks,ARI,Arizona
1,Atlanta Braves,ATL,Georgia
2,Baltimore Orioles,BAL,Maryland
3,Boston Red Sox,BOS,Massachusetts
4,Chicago Cubs,CHC,Illinois


Unamos esta dataframe con el de los equipos por estado

In [9]:
acronym_state = pd.merge(df_states, df_acronym, on = 'Estado')

In [10]:
acronym_state.head()

,Estado,Cantidad de equipos,Equipo,Acronimo
0,Arizona,1,Arizona Diamondbacks,ARI
1,California,5,Los Angeles Angels,LAA
2,California,5,Los Angeles Dodgers,LAD
3,California,5,Oakland Athletics,OAK
4,California,5,San Diego Padres,SD


En este caso, el nombre de las variables es claro

## Algoritmo para la creación de las bases de datos

A continuaicón, se optimizará el código para que se puedan obtener los *dataframes* anteriores para un cojuntos de datos de años secuenciales, como es nuestro caso

In [13]:
# Auxiliares:
free_agents = 'Data/Free_Agents/free_agents_'
hitting = 'Data/Statistics/Cumulative/Hitting/hitting_'
pitching = 'Data/Statistics/Cumulative/Pitching/pitching_'
salary = 'Data/Salary/salary_'
teams = 'ETL_Data/Agent/Teams/free_agents_team_'
csv = '.csv'
period = 12
# Originales:
df_free_agents = [None]*period
df_hitting = [None]*period
df_pitching = [None]*period
df_salary = [None]*period
df_teams = [None]*period
# Copias:
df_free_agents_copy = [None]*period
df_hitting_copy = [None]*period
df_pitching_copy = [None]*period
df_salary_copy = [None]*period
df_teams_copy = [None]*period
# Producto final:
df_pitchers = [None]*period
df_hitters = [None]*period
df_pitchers_free_agents = [None]*period
df_hitters_free_agents = [None]*period
df_pitchers_no_free_agents = [None]*period
df_hitters_no_free_agents = [None]*period
df_panel_hitters = [None]*period
df_panel_pitchers = [None]*period

Leamos todos los archivos y creemos las copias

In [14]:
for year in range(0,period):    
    df_free_agents[year] = pd.read_csv(free_agents + str(2011 + year) + csv)
    df_hitting[year] = pd.read_csv(hitting + str(2011 + year) + csv)
    df_pitching[year] = pd.read_csv(pitching + str(2011 + year) + csv)
    df_salary[year] = pd.read_csv(salary + str(2011 + year) + csv)
    df_teams[year] = pd.read_csv(teams + str(2011 + year) + csv)
    
    df_free_agents_copy[year] = df_free_agents[year].copy()
    df_hitting_copy[year] = df_hitting[year].copy()
    df_pitching_copy[year] = df_pitching[year].copy()
    df_salary_copy[year] = df_salary[year].copy()
    df_teams_copy[year] = pd.read_csv(teams + str(2011 + year) + csv)

Tratemos las bases de datos por separado. Sin embargo, a todas les quitaremos la columna de rango y *Cash2023*.

Como no queremos que se repita la columna del año de la temporada de la base de datos, borremos la columna de *Year* de la base  de datos de los agentes libres. Como los años del contrato aparecen en la base de datos sobre los salarios, se prefiere conservar dicha columna en la base de datos de salarios puesto que esta base de datos es más general que la de los agentes libres, razón por la que se borrará de esta última base de datos. 

El equipo al que se cambia el agente libre está señalado por la columna del equipo en la base de datos de salarios y la estadísticas deportivas por lo que se borrará *Team From To* de la base de datos de los agentes libres. 

Como nos importan los salarios para este analisis, quitaremos la columna de los equipos en las bases de datos sobre las estadísticas deportivas de todos los jugadores, así como la posición que ocupan.

In [15]:
for year in range(0,period):
    # Drop columns:
    if any(name in df_free_agents_copy[year].columns for name in ['Rank','Pos','Year','Team From To']):
        df_free_agents_copy[year].drop('Rank', axis = 1, inplace = True)
        df_free_agents_copy[year].drop('Year', axis = 1, inplace = True)
        df_free_agents_copy[year].drop('Pos', axis = 1, inplace = True)
        df_free_agents_copy[year].drop('Team From To', axis = 1, inplace = True)
    if 'Rank' in df_salary_copy[year].columns:
        df_salary_copy[year].drop('Rank', axis = 1, inplace = True)
    if any(name in df_hitting_copy[year].columns for name in ['Rank','Year','Cash2023','Team','Pos']):
        df_hitting_copy[year].drop('Rank', axis = 1, inplace = True)
        df_hitting_copy[year].drop('Cash2023', axis = 1, inplace = True)
        df_hitting_copy[year].drop('Team', axis = 1, inplace = True)
        df_hitting_copy[year].drop('Pos', axis = 1, inplace = True)
    if any(name in df_pitching_copy[year].columns for name in ['Rank','Year','Cash2023','Team','Pos']):
        df_pitching_copy[year].drop('Rank', axis = 1, inplace = True)
        df_pitching_copy[year].drop('Cash2023', axis = 1, inplace = True)
        df_pitching_copy[year].drop('Team', axis = 1, inplace = True)
        df_pitching_copy[year].drop('Pos', axis = 1, inplace = True)

Debido a que aparecen columnas que inician con el  nombre *Unnamed*, tendremos que borrarlas con algún método general, el cual se muestra a continuación:

In [16]:
for year in range(0,period):
    # Base de datos de agentes libres:
    df_free_agents_copy[year].drop(df_free_agents_copy[year].columns[df_free_agents_copy[year].columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)
    
    # Base de datos de los salarios:
    df_salary_copy[year].drop(df_salary_copy[year].columns[df_salary_copy[year].columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)
    
    # Base de datos de los hitters:
    df_hitting_copy[year].drop(df_hitting_copy[year].columns[df_hitting_copy[year].columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)
    
    # Base de datos de los pitchers:
    df_pitching_copy[year].drop(df_pitching_copy[year].columns[df_pitching_copy[year].columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)

Verifiquemos que ya no se encuentran dichas columnas molestas

In [17]:
df_free_agents_copy[9].columns

Index(['Player', 'Status', 'Team From', 'YRS', 'Value', 'AAV'], dtype='object')

In [18]:
df_salary_copy[11].columns

Index(['Player', 'Year', 'Pos', 'Team', 'BaseSalary', 'SigningBonus',
       'Payroll Salary', 'Adj Salary', 'Salary%', 'Cash', 'AAV', 'CONT YR',
       'CONT VALUE', 'Earnings', 'FA Year', 'Sign Age', 'Age', 'Weight',
       'Height'],
      dtype='object')

In [19]:
df_hitting_copy[2].columns

Index(['Player', 'GP', 'GP%', 'GS', 'GS%', 'AB', 'H', '2B', '3B', 'HR', 'RBI',
       'AVG', 'OBP', 'SLG', 'OPS', 'WAR', 'TVS'],
      dtype='object')

In [20]:
df_pitching_copy[5].columns

Index(['Player', 'GP', 'GS', 'IP', 'H', 'R', 'ER', 'BB', 'SO', 'W', 'L', 'SV',
       'WHIP', 'ERA', 'WAR', 'TVS'],
      dtype='object')

#### Agentes libres

No se conservará el equipo al que es contratado el agente libre puesto que esta información también la contiene la base de datos que facilita más el tratamiento _ETL_.

In [21]:
for year in range(0,period):
    df_free_agents_copy[year] = df_free_agents_copy[year].rename(columns = {'Player':'Jugador',
                                'Status':'Status_agente_libre', 'Team From':'Equipo_anterior',
                                'Value':'Valor_contrato', 'AAV':'Valor_promedio_contrato',
                                'YRS':'Anios_de_contrato'})
    
    free_agents_aux_1 = df_free_agents_copy[year]['Valor_contrato'].str.replace("$","")
    free_agents_aux_2 = free_agents_aux_1.str.replace(",","")
    free_agents_aux_3 = df_free_agents_copy[year]['Valor_promedio_contrato'].str.replace("$","")
    free_agents_aux_4 = free_agents_aux_3.str.replace(",","")
    df_free_agents_copy[year]['Valor_contrato'] = free_agents_aux_2
    df_free_agents_copy[year]['Valor_promedio_contrato'] = free_agents_aux_4
    
    df_free_agents_copy[year]['Valor_contrato'] = pd.to_numeric(df_free_agents_copy[year]['Valor_contrato'])
    df_free_agents_copy[year]['Valor_promedio_contrato'] = pd.to_numeric(df_free_agents_copy[year]['Valor_promedio_contrato'])

Observemos las dimensiones de las bases de datos como referencia

In [22]:
for year in range(0,period):
    print(df_free_agents_copy[year].shape)

(1, 6)
(108, 6)
(213, 6)
(208, 6)
(221, 6)
(241, 6)
(100, 6)
(98, 6)
(105, 6)
(118, 6)
(141, 6)
(137, 6)


También el tipo de datos que contiene cada columna

In [23]:
df_free_agents_copy[6].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Jugador                  100 non-null    object
 1   Status_agente_libre      100 non-null    object
 2   Equipo_anterior          100 non-null    object
 3   Anios_de_contrato        100 non-null    int64 
 4   Valor_contrato           100 non-null    int64 
 5   Valor_promedio_contrato  100 non-null    int64 
dtypes: int64(3), object(3)
memory usage: 4.8+ KB


Agreguemos los agentes libres en todas las temporadas que su contrato está vigente en lugar de solo tener observaciones en el año que firmaron. Para obervar cuántos datos se añadirán, veamos el tamaño inicial de las bases de datos.

In [24]:
period_t = period - 1
df_contracts = [None]*(period_t)

In [25]:
for year in range(1,period_t):
    
    max_year_contract = max(df_free_agents_copy[year]['Anios_de_contrato'])
    years = max_year_contract - 1
    df_contracts[year] = [None]*years
    
    for incremento in range(0,years):
        diff_t = 1 + incremento
        real_year = 2011 + year + diff_t
        year_bound = 2022

        if real_year <= year_bound:
            df_contracts[year][incremento] = df_free_agents_copy[year][df_free_agents_copy[year]['Anios_de_contrato'] > diff_t]

In [26]:
for year in range(1,period_t):
    years = len(df_contracts[year])
    
    for incremento in range(0,years):
        diff_t = 1 + incremento
        real_year = 2011 + year + diff_t
        year_bound = 2021

        if real_year <= year_bound:
            frames = [df_free_agents_copy[year + diff_t], df_contracts[year][incremento]]
            
            df_free_agents_copy[year + diff_t] = pd.concat(frames)
            
            df_free_agents_copy[year + diff_t].reset_index(drop = True, inplace = True)
            df_free_agents_copy[year + diff_t] = df_free_agents_copy[year + diff_t].sort_values(by = 'Jugador', ascending = True)

Veamos los resultados

In [27]:
for year in range(0,period):
    print(df_free_agents_copy[year].shape)

(1, 6)
(108, 6)
(243, 6)
(260, 6)
(293, 6)
(306, 6)
(182, 6)
(174, 6)
(189, 6)
(192, 6)
(203, 6)
(137, 6)


#### Salarios

Como los salarios irán con las bases de datos de los _hitters_ y _pitchers_ es que se hará su proceso _ETL_ antes.

In [28]:
for year in range(0,period):
    # Cambio de nombres
    df_salary_copy[year] = df_salary_copy[year].rename(columns = {'Player':'Jugador',
                            'BaseSalary':'Sueldo_base', 'SigningBonus':'Bono_por_firma',
                            'Payroll Salary':'Sueldo_regular', 'Adj Salary':'Sueldo_ajustado',
                            'CONT YR':'Anios_de_contrato', 'CONT VALUE':'Valor_del_contrato',
                            'Earnings':'Ganancias', 'FA Year':'Anio_de_agente_libre',
                            'Sign Age':'Edad_al_firmar', 'Age':'Edad', 'Weight':'Peso',
                            'Height':'Altura', 'Year':'Anio', 'Pos':'Posicion',
                            'Salary%':'Sueldo_porcentual', 'Cash':'Pago_efectivo',
                            'AAV':'Valor_contrato_promedio', 'Team':'Acronimo'})
    
    # Tranformando al tipo de dato apropiado
    salary_aux_1 = df_salary_copy[year]['Sueldo_base'].str.replace("$","")
    salary_aux_2 = salary_aux_1.str.replace(",","")
    df_salary_copy[year]['Sueldo_base'] = salary_aux_2
    df_salary_copy[year]['Sueldo_base'] = pd.to_numeric(df_salary_copy[year]['Sueldo_base'])
    
    salary_aux_3 = df_salary_copy[year]['Sueldo_regular'].str.replace("$","")
    salary_aux_4 = salary_aux_3.str.replace(",","")
    df_salary_copy[year]['Sueldo_regular'] = salary_aux_4
    df_salary_copy[year]['Sueldo_regular'] = pd.to_numeric(df_salary_copy[year]['Sueldo_regular'])
    
    salary_aux_5 = df_salary_copy[year]['Sueldo_ajustado'].str.replace("$","")
    salary_aux_6 = salary_aux_5.str.replace(",","")
    df_salary_copy[year]['Sueldo_ajustado'] = salary_aux_6
    df_salary_copy[year]['Sueldo_ajustado'] = pd.to_numeric(df_salary_copy[year]['Sueldo_ajustado'])
    
    salary_aux_7 = df_salary_copy[year]['Valor_del_contrato'].str.replace("$","")
    salary_aux_8 = salary_aux_7.str.replace(",","")
    df_salary_copy[year]['Valor_del_contrato'] = salary_aux_8
    df_salary_copy[year]['Valor_del_contrato'] = pd.to_numeric(df_salary_copy[year]['Valor_del_contrato'])
    
    salary_aux_9 = df_salary_copy[year]['Bono_por_firma'].str.replace("$","")
    salary_aux_10 = salary_aux_9.str.replace(",","")
    df_salary_copy[year]['Bono_por_firma'] = salary_aux_10
    df_salary_copy[year]['Bono_por_firma'] = pd.to_numeric(df_salary_copy[year]['Bono_por_firma'])
    
    salary_aux_11 = df_salary_copy[year]['Ganancias'].str.replace("$","")
    salary_aux_12 = salary_aux_11.str.replace(",","")
    df_salary_copy[year]['Ganancias'] = salary_aux_12
    df_salary_copy[year]['Ganancias'] = pd.to_numeric(df_salary_copy[year]['Ganancias'])
    
    salary_aux_13 = df_salary_copy[year]['Pago_efectivo'].str.replace("$","")
    salary_aux_14 = salary_aux_13.str.replace(",","")
    df_salary_copy[year]['Pago_efectivo'] = salary_aux_14
    df_salary_copy[year]['Pago_efectivo'] = pd.to_numeric(df_salary_copy[year]['Pago_efectivo'])
    
    salary_aux_15 = df_salary_copy[year]['Valor_contrato_promedio'].str.replace("$","")
    salary_aux_16 = salary_aux_15.str.replace(",","")
    df_salary_copy[year]['Valor_contrato_promedio'] = salary_aux_16
    df_salary_copy[year]['Valor_contrato_promedio'] = pd.to_numeric(df_salary_copy[year]['Valor_contrato_promedio'])
    
    salary_aux_17 = df_salary_copy[year]['Altura'].str.replace("\"","")
    salary_aux_18 = salary_aux_17.str.replace("'","")
    df_salary_copy[year]['Altura'] = salary_aux_18
    df_salary_copy[year]['Altura'] = pd.to_numeric(df_salary_copy[year]['Altura'])/10
    # SUstitullamos los xeros
    height_mean = df_salary_copy[year]['Altura'].mean(skipna=True)
    df_salary_copy[year]['Altura'] = df_salary_copy[year].Altura.mask(df_salary_copy[year].Altura == 0, height_mean)
    
    df_salary_copy[year]['Anio_de_agente_libre'] = pd.to_numeric(df_salary_copy[year]['Anio_de_agente_libre'])
    df_salary_copy[year]['Anios_de_contrato'] = pd.to_numeric(df_salary_copy[year]['Anios_de_contrato'])
    df_salary_copy[year]['Edad'] = pd.to_numeric(df_salary_copy[year]['Edad'])

Por algunas particularidades de la base de datos, las columna que contiene la edad al firmar se tratará por separado aprovechando que la mayoría de los datos incorrectos tienen una longitud mayor a dos.

In [29]:
for year in range (0,period):
    df_salary_copy[year]['Edad_al_firmar'] = df_salary_copy[year]['Edad_al_firmar'].map(str)

    for edad in range(0,df_salary_copy[year].shape[0]):
        # String es mayor que 0:
        if len(df_salary_copy[year]['Edad_al_firmar'].iloc[edad]) == 2:
            df_salary_copy[year].iloc[edad, df_salary_copy[year].columns.get_loc('Edad_al_firmar')] = pd.to_numeric(df_salary_copy[year]['Edad_al_firmar'].iloc[edad])
            
        # String es menor o igual que 0:
        elif len(df_salary_copy[year]['Edad_al_firmar'].iloc[edad]) != 2:
            # Si la columna de la edad contiene datos correctos
            if df_salary_copy[year]['Edad'].iloc[edad] > 0:
                if df_salary_copy[year]['Anio_de_agente_libre'].iloc[edad] == 0:
                    ag_year = year + 2011 + 1
                else:
                    ag_year = df_salary_copy[year]['Anio_de_agente_libre'].iloc[edad]
                # Get first year of contract
                ini_year = ag_year - df_salary_copy[year]['Anios_de_contrato'].iloc[edad]
                # Años desde el el año inicial
                dif_years = year + 2011 - ini_year
                # Edad al firmar:
                sign_age = df_salary_copy[year]['Edad'].iloc[edad] - dif_years
                # Cambio de dato:
                df_salary_copy[year].iloc[edad, df_salary_copy[year].columns.get_loc('Edad_al_firmar')] = pd.to_numeric(sign_age)
                
            # Si la columna de edad no contiene un dato coherente
            else:
                # Cambio de dato:
                df_salary_copy[year].iloc[edad, df_salary_copy[year].columns.get_loc('Edad_al_firmar')] = pd.to_numeric(18)    
    
        # Entero  menor a 0:
        if df_salary_copy[year]['Edad_al_firmar'].iloc[edad] < 0:
            # Si la columna de la edad contiene datos correctos
            if df_salary_copy[year]['Edad'].iloc[edad] > 0:
                if df_salary_copy[year]['Anio_de_agente_libre'].iloc[edad] == 0:
                    ag_year = year + 2011 + 1
                else:
                    ag_year = df_salary_copy[year]['Anio_de_agente_libre'].iloc[edad]
                # Get first year of contract
                ini_year = ag_year - df_salary_copy[year]['Anios_de_contrato'].iloc[edad]
                # Años desde el el año inicial
                dif_years = year + 2011 - ini_year
                # Edad al firmar:
                sign_age = df_salary_copy[year]['Edad'].iloc[edad] - dif_years
                # Cambio de dato:
                df_salary_copy[year].iloc[edad, df_salary_copy[year].columns.get_loc('Edad_al_firmar')] = pd.to_numeric(sign_age)
                
            # Si la columna de edad no contiene un dato coherente
            else:
                # Cambio de dato:
                df_salary_copy[year].iloc[edad, df_salary_copy[year].columns.get_loc('Edad_al_firmar')] = pd.to_numeric(18)
         
    # Transformemos los datos a enteros
    df_salary_copy[year]['Edad_al_firmar'] = pd.to_numeric(df_salary_copy[year]['Edad_al_firmar'])

Podemos verificar si se limpiaron adecuadamente las celdas de la columna de edades al firmar. Esto, al filtrar los datos que sean distintos a enteros y al observar si se pudo transformar toda la columna al tipo entero.

In [30]:
for year in range(0,period):
    for edad in range(0,df_salary_copy[year]['Edad_al_firmar'].shape[0]):
        if type(df_salary_copy[year]['Edad_al_firmar'].iloc[edad]) != np.int64:
            print(type(df_salary_copy[year]['Edad_al_firmar'].iloc[edad]))

In [31]:
for year in range(0,period):
    for edad in range(0,df_salary_copy[year]['Edad_al_firmar'].shape[0]):
        if df_salary_copy[year]['Edad_al_firmar'].iloc[edad] < 0:
            print(df_salary_copy[year]['Edad_al_firmar'].iloc[edad])

In [32]:
#for year in range(0,period):
#    print(type(df_salary_copy[year][['Edad_al_firmar']].info()))

In [33]:
#for year in range(0,period):
#    print(year)
#    for edad in range(0,df_salary_copy[year]['Edad_al_firmar'].shape[0]):
#        print(str(df_salary_copy[year]['Edad_al_firmar'].iloc[edad]) + ' ' + str(edad))

Por otro lado, falta corregir las entradas de las columnas de las edades que tengan valores menores a cero. Esto se hará de acuerdo al resto de columnas

In [34]:
for year in range(0,period):
    for edad in range(0,df_salary_copy[year]['Edad'].shape[0]):
        if df_salary_copy[year]['Edad'].iloc[edad] < 0:
            print(year)
            print(df_salary_copy[year]['Edad'].iloc[edad])

0
-9
1
-8
2
-7


In [35]:
for year in range(0,period):
    for edad in range(0,df_salary_copy[year].shape[0]):
        # Condición para imputar:
        if df_salary_copy[year]['Edad'].iloc[edad] <= 0:
            # Si no se indica si tendrá año de agencia libre:
            if df_salary_copy[year]['Anio_de_agente_libre'].iloc[edad] == 0:
                        ag_year = year + 2011 + 1
            # Si tendrá año de agencia libre
            else:
                ag_year = df_salary_copy[year]['Anio_de_agente_libre'].iloc[edad]
            # Get first year of contract
            ini_year = ag_year - df_salary_copy[year]['Anios_de_contrato'].iloc[edad]
            # Años desde el el año inicial
            dif_years = year + 2011 - ini_year
            # Edad en la temporada:
            seasson_age = df_salary_copy[year]['Edad_al_firmar'].iloc[edad] + dif_years
            # Asignación
            df_salary_copy[year]['Edad'].iloc[edad] = seasson_age

Comprobemos que no hay ninguna edad negativa

In [36]:
for year in range(0,period):
    for edad in range(0,df_salary_copy[year]['Edad'].shape[0]):
        if df_salary_copy[year]['Edad'].iloc[edad] < 0:
            print(year)
            print(str(df_salary_copy[year]['Edad'].iloc[edad]) + ' ' + str(edad))

Con la imputación de datos ya se puede crear la columna que contiene la antiguedad del agente libre bajo el contrato

In [37]:
for year in range(0,period):
    df_salary_copy[year]['Antiguedad'] = df_salary_copy[year]['Edad'] - df_salary_copy[year]['Edad_al_firmar']

Por último, convirtamos la columna del año a string para que se entienda como una categoría y no una variable numérica

In [38]:
for year in range(0,period):
    df_salary_copy[year]['Anio'] = df_salary_copy[year]['Anio'].map(str)

In [39]:
df_salary_copy[5].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1045 entries, 0 to 1044
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Jugador                  1045 non-null   object 
 1   Anio                     1045 non-null   object 
 2   Posicion                 1045 non-null   object 
 3   Acronimo                 1045 non-null   object 
 4   Sueldo_base              1045 non-null   int64  
 5   Bono_por_firma           1045 non-null   int64  
 6   Sueldo_regular           1045 non-null   int64  
 7   Sueldo_ajustado          1045 non-null   int64  
 8   Sueldo_porcentual        1045 non-null   float64
 9   Pago_efectivo            1045 non-null   int64  
 10  Valor_contrato_promedio  1045 non-null   int64  
 11  Anios_de_contrato        1045 non-null   int64  
 12  Valor_del_contrato       1045 non-null   int64  
 13  Ganancias                1045 non-null   int64  
 14  Anio_de_agente_libre    

#### Hitters

In [40]:
for year in range(0,period):
    # Cambio de nombres
    df_hitting_copy[year] = df_hitting_copy[year].rename(columns = {'Player':'Jugador',
                            'GP':'Juegos', 'GP%':'Porcentaje_juegos',
                            'AB':'At-bats', 'H':'Bateos', 'GS':'Juegos_iniciados',
                            'GS%':'Porcentaje_juegos_iniciados', 'RBI':'Runs-batted-in',
                            'HR':'Home-runs', 'AVG':'Bateos_promedio',
                            '2B':'Dobles', '3B':'Triples', 'OPS':'Porcentaje_On-base-plus-slugging',
                            'SLG':'Porcentaje_slugging', 'OBP':'Porcentaje_on-base'})

In [41]:
for year in range(0,period):
    print(df_hitting_copy[year].shape)

(521, 17)
(667, 17)
(734, 17)
(778, 17)
(809, 17)
(834, 17)
(810, 17)
(825, 17)
(850, 17)
(451, 17)
(753, 17)
(479, 17)


In [42]:
df_hitting_copy[5].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 834 entries, 0 to 833
Data columns (total 17 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Jugador                           834 non-null    object 
 1   Juegos                            834 non-null    int64  
 2   Porcentaje_juegos                 834 non-null    float64
 3   Juegos_iniciados                  834 non-null    int64  
 4   Porcentaje_juegos_iniciados       834 non-null    float64
 5   At-bats                           834 non-null    int64  
 6   Bateos                            834 non-null    int64  
 7   Dobles                            834 non-null    int64  
 8   Triples                           834 non-null    int64  
 9   Home-runs                         834 non-null    int64  
 10  Runs-batted-in                    834 non-null    int64  
 11  Bateos_promedio                   834 non-null    float64
 12  Porcenta

In [43]:
df_hitting_copy[5].columns

Index(['Jugador', 'Juegos', 'Porcentaje_juegos', 'Juegos_iniciados',
       'Porcentaje_juegos_iniciados', 'At-bats', 'Bateos', 'Dobles', 'Triples',
       'Home-runs', 'Runs-batted-in', 'Bateos_promedio', 'Porcentaje_on-base',
       'Porcentaje_slugging', 'Porcentaje_On-base-plus-slugging', 'WAR',
       'TVS'],
      dtype='object')

#### Pitchers

In [44]:
for year in range(0,period):
    # Cambio de nombres
    df_pitching_copy[year] = df_pitching_copy[year].rename(columns = {'Player':'Jugador',
                             'GP':'Juegos', 'GS':'Juegos_iniciados', 'IP':'Inning_pitched',
                             'H':'Bateos', 'R':'Carreras', 'ER':'Carreras_ganadas',
                             'BB':'Walks', 'SO':'Strike-outs', 'W':'Wins', 'L':'Losses',
                             'SV':'Saves'})

In [45]:
for year in range(0,period):
    print(df_pitching_copy[year].shape)

(324, 16)
(427, 16)
(485, 16)
(537, 16)
(586, 16)
(592, 16)
(581, 16)
(644, 16)
(668, 16)
(542, 16)
(611, 16)
(598, 16)


In [46]:
df_pitching_copy[5].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 592 entries, 0 to 591
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Jugador           592 non-null    object 
 1   Juegos            592 non-null    int64  
 2   Juegos_iniciados  592 non-null    int64  
 3   Inning_pitched    592 non-null    float64
 4   Bateos            592 non-null    int64  
 5   Carreras          592 non-null    int64  
 6   Carreras_ganadas  592 non-null    int64  
 7   Walks             592 non-null    int64  
 8   Strike-outs       592 non-null    int64  
 9   Wins              592 non-null    int64  
 10  Losses            592 non-null    int64  
 11  Saves             592 non-null    int64  
 12  WHIP              592 non-null    float64
 13  ERA               592 non-null    float64
 14  WAR               587 non-null    float64
 15  TVS               592 non-null    float64
dtypes: float64(5), int64(10), object(1)
memory u

## Agregación de variables sugeridas por artículos

Las primeras variables que agregaremos son el cuadrado de todas las estadísticas deportivas, así como las siguientes variables:

- DOMINANCE = $Strike-outs/(9*Inning \; Pitched)$
- CONTROL = $Walks/(9*Inning \; Pitched)$
- COMMAND = $Strike-outs/Walks$

In [47]:
for year in range(0,period):
    df_pitching_copy[year]['Dominio'] = df_pitching_copy[year]['Strike-outs']/(9*df_pitching_copy[year]['Inning_pitched'])
    df_pitching_copy[year]['Control'] = df_pitching_copy[year]['Walks']/(9*df_pitching_copy[year]['Inning_pitched'])
    df_pitching_copy[year]['Comando'] = df_pitching_copy[year]['Strike-outs']/df_pitching_copy[year]['Walks']

In [48]:
for year in range(0,period):
    print(df_pitching_copy[year].shape)

(324, 19)
(427, 19)
(485, 19)
(537, 19)
(586, 19)
(592, 19)
(581, 19)
(644, 19)
(668, 19)
(542, 19)
(611, 19)
(598, 19)


In [49]:
df_pitching_copy[2].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 485 entries, 0 to 484
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Jugador           485 non-null    object 
 1   Juegos            485 non-null    int64  
 2   Juegos_iniciados  485 non-null    int64  
 3   Inning_pitched    485 non-null    float64
 4   Bateos            485 non-null    int64  
 5   Carreras          485 non-null    int64  
 6   Carreras_ganadas  485 non-null    int64  
 7   Walks             485 non-null    int64  
 8   Strike-outs       485 non-null    int64  
 9   Wins              485 non-null    int64  
 10  Losses            485 non-null    int64  
 11  Saves             485 non-null    int64  
 12  WHIP              485 non-null    float64
 13  ERA               485 non-null    float64
 14  WAR               435 non-null    float64
 15  TVS               485 non-null    float64
 16  Dominio           484 non-null    float64
 1

Podemos verificar qué entradas poseen valores infinitos en la base de datos

In [50]:
"""
for year in range(0,period):
    print(str(2011 + year))
    for name in df_pitching_copy[year].columns:
        print(name)
        if type(name) != str:
            for element in range(0,len(df_pitching_copy[year][name])):
                if math.isinf(df_pitching_copy[year][name].iloc[element]) == True:
                    print(str(element) +  '  ' + str(df_pitching_copy[year][name].iloc[element]))
    print("")
"""

'\nfor year in range(0,period):\n    print(str(2011 + year))\n    for name in df_pitching_copy[year].columns:\n        print(name)\n        if type(name) != str:\n            for element in range(0,len(df_pitching_copy[year][name])):\n                if math.isinf(df_pitching_copy[year][name].iloc[element]) == True:\n                    print(str(element) +  \'  \' + str(df_pitching_copy[year][name].iloc[element]))\n    print("")\n'

Siguiendo la sugerencia de algunos artículos, obtengamos el logaritmo de los salarios

In [51]:
for year in range(0,period):
    df_salary_copy[year]['ln_Sueldo_base'] = np.log(df_salary_copy[year]['Sueldo_base'])
    df_salary_copy[year]['ln_Sueldo_ajustado'] = np.log(df_salary_copy[year]['Sueldo_ajustado'])
    df_salary_copy[year]['ln_Sueldo_regular'] = np.log(df_salary_copy[year]['Sueldo_regular'])

In [52]:
df_salary_copy[2].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1011 entries, 0 to 1010
Data columns (total 23 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Jugador                  1011 non-null   object 
 1   Anio                     1011 non-null   object 
 2   Posicion                 1011 non-null   object 
 3   Acronimo                 1011 non-null   object 
 4   Sueldo_base              1011 non-null   int64  
 5   Bono_por_firma           1011 non-null   int64  
 6   Sueldo_regular           1011 non-null   int64  
 7   Sueldo_ajustado          1011 non-null   int64  
 8   Sueldo_porcentual        1011 non-null   float64
 9   Pago_efectivo            1011 non-null   int64  
 10  Valor_contrato_promedio  1011 non-null   int64  
 11  Anios_de_contrato        1011 non-null   int64  
 12  Valor_del_contrato       1011 non-null   int64  
 13  Ganancias                1011 non-null   int64  
 14  Anio_de_agente_libre    

Debido a que hay columnas con datos tipo _Nan_ o _NULL_, optaremos por imputarlos.

Mientras que los valores infinitos generados por las nuevas variables se sustituirán dependediendo del caso:

- 0/0: 0
- num/0: Máximo de la columna correspondientefijarán

In [53]:
for year in range(0,period):
    # Salaries
    mean_hgt = df_salary_copy[year].loc[df_salary_copy[year]['Altura'] > 4.9].Altura.mean()
    mean_wgh = df_salary_copy[year].loc[df_salary_copy[year]['Peso'] > 0].Peso.mean()
    df_salary_copy[year]['Altura'].fillna(value = mean_hgt, inplace = True)
    df_salary_copy[year]['Altura'].mask(df_salary_copy[year]['Altura'] <= 4.9, mean_hgt, inplace = True)
    df_salary_copy[year]['Peso'].fillna(value = mean_wgh, inplace = True)
    df_salary_copy[year]['Peso'].mask(df_salary_copy[year]['Peso'] <= 0, mean_wgh, inplace = True)
    
    # Pitchers
    mean_war = df_pitching_copy[year].loc[df_pitching_copy[year]['WAR'] > 0].WAR.mean()
    mean_dom = df_pitching_copy[year].loc[df_pitching_copy[year]['Dominio'] > 0].Dominio.mean()
    mean_con = df_pitching_copy[year].loc[df_pitching_copy[year]['Control'] > 0].Control.mean()
    mean_com = df_pitching_copy[year].loc[df_pitching_copy[year]['Comando'] > 0].Comando.mean()
    df_pitching_copy[year]['WAR'].fillna(value = mean_war, inplace = True)
    df_pitching_copy[year]['WAR'].mask(df_pitching_copy[year]['WAR'] <= 0, mean_war, inplace = True)
    df_pitching_copy[year]['Dominio'].fillna(value = mean_dom, inplace = True)
    df_pitching_copy[year]['Dominio'].mask(df_pitching_copy[year]['Dominio'] <= 0, mean_dom, inplace = True)
    df_pitching_copy[year]['Control'].fillna(value = mean_con, inplace = True)
    df_pitching_copy[year]['Control'].mask(df_pitching_copy[year]['Control'] <= 0, mean_con, inplace = True)
    df_pitching_copy[year]['Comando'].fillna(value = mean_com, inplace = True)
    df_pitching_copy[year]['Comando'].mask(df_pitching_copy[year]['Comando'] <= 0, mean_com, inplace = True)
    
    # Hitters
    mean_war = df_hitting_copy[year].loc[df_hitting_copy[year]['WAR'] > 0].WAR.mean()
    df_hitting_copy[year]['WAR'].fillna(value = mean_war, inplace = True)
    df_hitting_copy[year]['WAR'].mask(df_hitting_copy[year]['WAR'] <= 0, mean_war, inplace = True)

In [54]:
for year in range(0,period):   
    # Condiciones
    con_dom_1 = df_pitching_copy[year]['Strike-outs'] == 0
    con_con_1 = df_pitching_copy[year]['Walks'] == 0
    con_com_1 = df_pitching_copy[year]['Strike-outs'] == 0
                 
    # Imputación caso 0/0
    df_pitching_copy[year].loc[con_dom_1, "Dominio"] = 0
    df_pitching_copy[year].loc[con_con_1, "Control"] = 0
    df_pitching_copy[year].loc[con_com_1, "Comando"] = 0

In [55]:
for year in range(0,period):   
    # Máximos
    max_dom = df_pitching_copy[year]['Strike-outs'].max()/9
    max_con = df_pitching_copy[year]['Walks'].max()/9
    max_com = df_pitching_copy[year]['Strike-outs'].max()
    
    # Cambianfdo infinitos a NaNs
    df_pitching_copy[year]["Dominio"].replace([np.inf, -np.inf], np.nan, inplace = True)
    df_pitching_copy[year]["Control"].replace([np.inf, -np.inf], np.nan, inplace = True)
    df_pitching_copy[year]["Comando"].replace([np.inf, -np.inf], np.nan, inplace = True)
    
    # Imputación
    df_pitching_copy[year]['Dominio'].fillna(value = max_dom, inplace = True)
    df_pitching_copy[year]['Control'].fillna(value = max_con, inplace = True)
    df_pitching_copy[year]['Comando'].fillna(value = max_com, inplace = True)

Verifiquemos que ya no haya problemas con valores infinitos

In [56]:
"""
for year in range(0,period):
    print(str(2011 + year))
    for name in df_pitching_copy[year].columns:
        print(name)
        if type(name) != str:
            for element in range(0,len(df_pitching_copy[year][name])):
                if math.isinf(df_pitching_copy[year][name].iloc[element]) == True:
                    print(str(element) +  '  ' + str(df_pitching_copy[year][name].iloc[element]))
    print("")
"""

'\nfor year in range(0,period):\n    print(str(2011 + year))\n    for name in df_pitching_copy[year].columns:\n        print(name)\n        if type(name) != str:\n            for element in range(0,len(df_pitching_copy[year][name])):\n                if math.isinf(df_pitching_copy[year][name].iloc[element]) == True:\n                    print(str(element) +  \'  \' + str(df_pitching_copy[year][name].iloc[element]))\n    print("")\n'

Así mismo, contemos los valores *NaN* que queden presentes

In [57]:
for year in range(0,period):
    print('Año: ' + str(2011 + year))
    print('Hitters:')
    df_hitting_copy[year].isna().sum()
    print('Pitchers:')
    df_pitching_copy[year].isna().sum()
    print('Free agents:')
    df_free_agents_copy[year].isna().sum()
    print('Salaries:')
    df_salary_copy[year].isna().sum()
    print("")

Año: 2011
Hitters:
Pitchers:
Free agents:
Salaries:

Año: 2012
Hitters:
Pitchers:
Free agents:
Salaries:

Año: 2013
Hitters:
Pitchers:
Free agents:
Salaries:

Año: 2014
Hitters:
Pitchers:
Free agents:
Salaries:

Año: 2015
Hitters:
Pitchers:
Free agents:
Salaries:

Año: 2016
Hitters:
Pitchers:
Free agents:
Salaries:

Año: 2017
Hitters:
Pitchers:
Free agents:
Salaries:

Año: 2018
Hitters:
Pitchers:
Free agents:
Salaries:

Año: 2019
Hitters:
Pitchers:
Free agents:
Salaries:

Año: 2020
Hitters:
Pitchers:
Free agents:
Salaries:

Año: 2021
Hitters:
Pitchers:
Free agents:
Salaries:

Año: 2022
Hitters:
Pitchers:
Free agents:
Salaries:



Ahora, repitamos este proceso para la base de datos de los salarios.

In [58]:
salary_names = ['ln_Sueldo_ajustado', 'ln_Sueldo_base', 'ln_Sueldo_regular']

In [59]:
for name in salary_names:
    print(name)
    
    for year in range(0,period):
        print(str(2011 + year))
        for element in range(0,len(df_salary_copy[year][name])):
            if df_salary_copy[year][name].iloc[element] <= 0:
                print(str(element) +  '  ' + str(df_salary_copy[year][name].iloc[element]))
        print("")

ln_Sueldo_ajustado
2011

2012

2013

2014

2015

2016

2017

2018
72  -inf
188  -inf

2019

2020

2021
6  -inf
56  -inf
166  -inf
274  -inf

2022
201  -inf

ln_Sueldo_base
2011

2012

2013

2014

2015

2016

2017

2018

2019

2020

2021

2022

ln_Sueldo_regular
2011

2012

2013

2014

2015

2016

2017

2018

2019

2020

2021

2022



Al inspecsionar los errores nos damos cuenta que solo se desconocen los salarios ajustados y los fijaron a $0$. Usaremos el logaritmo base 10 del salario regular para sustituir dicho valor.

In [60]:
for year in range(0,period):
    df_salary_copy[year]['ln_Sueldo_ajustado'].mask(df_salary_copy[year]['ln_Sueldo_ajustado'] < 0,
                                                    df_salary_copy[year]['ln_Sueldo_regular'],
                                                    inplace = True)

In [61]:
for year in range(0,period):
    print(str(2011 + year))
    for element in range(0,len(df_salary_copy[year]['ln_Sueldo_ajustado'])):
        if df_salary_copy[year]['ln_Sueldo_ajustado'].iloc[element] <= 0:
            print(str(element) +  '  ' + str(df_salary_copy[year]['ln_Sueldo_ajustado'].iloc[element]))
    print("")

2011

2012

2013

2014

2015

2016

2017

2018

2019

2020

2021

2022



In [62]:
for year in range(0,period):
    print("Ajustado: " + str(df_salary_copy[year]['ln_Sueldo_ajustado'].mean())
          + 'n'
          + 'Regular: ' + str(df_salary_copy[year]['ln_Sueldo_regular'].mean()))

Ajustado: 13.786883876822797nRegular: 13.888874386883515
Ajustado: 13.836339875004654nRegular: 13.909807816176343
Ajustado: 13.547035081556775nRegular: 13.9779431913153
Ajustado: 13.66411021920866nRegular: 14.080738990191257
Ajustado: 13.49415866539551nRegular: 13.99344595993313
Ajustado: 13.478992231977141nRegular: 14.0261405412466
Ajustado: 13.575627559754176nRegular: 14.080502234724484
Ajustado: 13.562343599267946nRegular: 14.08078787299267
Ajustado: 13.52136464369358nRegular: 14.066559580453122
Ajustado: 13.029933516088823nRegular: 14.217470030278857
Ajustado: 13.688221153044282nRegular: 14.18813695050661
Ajustado: 13.923634888060981nRegular: 14.351533324360805


En efecto, ya no hay valores _NaN_ o _infinitos_.

Con el objetivo de hacer más eficiente la creación de las variables al cuadrado, lo haremos extrayendo el índice de las columnas de interés

In [63]:
df_hitting_copy[0].columns

Index(['Jugador', 'Juegos', 'Porcentaje_juegos', 'Juegos_iniciados',
       'Porcentaje_juegos_iniciados', 'At-bats', 'Bateos', 'Dobles', 'Triples',
       'Home-runs', 'Runs-batted-in', 'Bateos_promedio', 'Porcentaje_on-base',
       'Porcentaje_slugging', 'Porcentaje_On-base-plus-slugging', 'WAR',
       'TVS'],
      dtype='object')

In [64]:
df_pitching_copy[1].columns

Index(['Jugador', 'Juegos', 'Juegos_iniciados', 'Inning_pitched', 'Bateos',
       'Carreras', 'Carreras_ganadas', 'Walks', 'Strike-outs', 'Wins',
       'Losses', 'Saves', 'WHIP', 'ERA', 'WAR', 'TVS', 'Dominio', 'Control',
       'Comando'],
      dtype='object')

In [65]:
def get_col_indices(df, names):
    return df.columns.get_indexer(names)

In [66]:
hitting_names = ['Juegos_iniciados', 'Porcentaje_juegos_iniciados', 'At-bats', 'Bateos',
                  'Dobles', 'Triples', 'Home-runs', 'Runs-batted-in', 'Bateos_promedio',
                  'Porcentaje_on-base', 'Porcentaje_slugging', 'TVS',
                  'Porcentaje_On-base-plus-slugging', 'WAR']	
pitching_names = ['Inning_pitched', 'Bateos', 'Carreras',
                  'Carreras_ganadas', 'Walks', 'Strike-outs', 'Wins', 'Losses',
                  'Saves', 'WHIP', 'ERA', 'WAR', 'TVS', 'Dominio', 'Control',
                  'Comando']

Con el objetivo de simplificar el código, verifiquemos si todos los índices en cada base de datos son los mismos

In [67]:
print('Hitters:')
for year in range(0,period):
    print(get_col_indices(df_hitting_copy[year], hitting_names))
    
print('Pitchers:')
for year in range(0,period):
    print(get_col_indices(df_pitching_copy[year], pitching_names))

Hitters:
[ 3  4  5  6  7  8  9 10 11 12 13 16 14 15]
[ 3  4  5  6  7  8  9 10 11 12 13 16 14 15]
[ 3  4  5  6  7  8  9 10 11 12 13 16 14 15]
[ 3  4  5  6  7  8  9 10 11 12 13 16 14 15]
[ 3  4  5  6  7  8  9 10 11 12 13 16 14 15]
[ 3  4  5  6  7  8  9 10 11 12 13 16 14 15]
[ 3  4  5  6  7  8  9 10 11 12 13 16 14 15]
[ 3  4  5  6  7  8  9 10 11 12 13 16 14 15]
[ 3  4  5  6  7  8  9 10 11 12 13 16 14 15]
[ 3  4  5  6  7  8  9 10 11 12 13 16 14 15]
[ 3  4  5  6  7  8  9 10 11 12 13 16 14 15]
[ 3  4  5  6  7  8  9 10 11 12 13 16 14 15]
Pitchers:
[ 3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18]
[ 3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18]
[ 3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18]
[ 3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18]
[ 3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18]
[ 3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18]
[ 3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18]
[ 3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18]
[ 3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18]
[ 3

In [68]:
hitting_indexes = list(get_col_indices(df_hitting_copy[0], hitting_names))
pitching_indexes = list(get_col_indices(df_pitching_copy[0], pitching_names))

In [69]:
for year in range(0,period):
    # Hitters:
    for hitter_name in hitting_indexes:
        df_hitting_copy[year][df_hitting_copy[year].columns[hitter_name] + '_2'] = np.power(df_hitting_copy[year][df_hitting_copy[year].columns[hitter_name]], 2)
    # Pitchers:
    for pitcher_name in pitching_indexes:
        df_pitching_copy[year][df_pitching_copy[year].columns[pitcher_name] + '_2'] = np.power(df_pitching_copy[year][df_pitching_copy[year].columns[pitcher_name]], 2)

Apreciemos el resultado final

In [70]:
df_hitting_copy[2].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 734 entries, 0 to 733
Data columns (total 31 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Jugador                             734 non-null    object 
 1   Juegos                              734 non-null    int64  
 2   Porcentaje_juegos                   734 non-null    float64
 3   Juegos_iniciados                    734 non-null    int64  
 4   Porcentaje_juegos_iniciados         734 non-null    float64
 5   At-bats                             734 non-null    int64  
 6   Bateos                              734 non-null    int64  
 7   Dobles                              734 non-null    int64  
 8   Triples                             734 non-null    int64  
 9   Home-runs                           734 non-null    int64  
 10  Runs-batted-in                      734 non-null    int64  
 11  Bateos_promedio                     734 non-n

In [71]:
df_pitching_copy[2].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 485 entries, 0 to 484
Data columns (total 35 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Jugador             485 non-null    object 
 1   Juegos              485 non-null    int64  
 2   Juegos_iniciados    485 non-null    int64  
 3   Inning_pitched      485 non-null    float64
 4   Bateos              485 non-null    int64  
 5   Carreras            485 non-null    int64  
 6   Carreras_ganadas    485 non-null    int64  
 7   Walks               485 non-null    int64  
 8   Strike-outs         485 non-null    int64  
 9   Wins                485 non-null    int64  
 10  Losses              485 non-null    int64  
 11  Saves               485 non-null    int64  
 12  WHIP                485 non-null    float64
 13  ERA                 485 non-null    float64
 14  WAR                 485 non-null    float64
 15  TVS                 485 non-null    float64
 16  Dominio 

## Unión de las bases de datos
### Datos agregados por equipo

Solo resta añadir los datos relevantes al equipo al que pertenece cada jugador considerando la base de datos de la cantidad de equipos por estado

In [72]:
df_teams_copy[7].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 0 to 27
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Equipo                   28 non-null     object 
 1   Cantidad_agentes_libres  28 non-null     int64  
 2   Valor_contrato_total     28 non-null     int64  
 3   Acronimo                 28 non-null     object 
 4   Victorias                28 non-null     int64  
 5   Juegos totales           28 non-null     int64  
 6   Playoffs                 28 non-null     int64  
 7   Pennants won             28 non-null     int64  
 8   WS ganadas               28 non-null     int64  
 9   Promedio_victorias       28 non-null     float64
dtypes: float64(1), int64(7), object(2)
memory usage: 2.3+ KB


In [73]:
acronym_state.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30 entries, 0 to 29
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Estado               30 non-null     object
 1   Cantidad de equipos  30 non-null     int64 
 2   Equipo               30 non-null     object
 3   Acronimo             30 non-null     object
dtypes: int64(1), object(3)
memory usage: 1.2+ KB


In [74]:
for year in range(0,period):
    df_teams_copy[year] = pd.merge(df_teams_copy[year], acronym_state, on = ['Equipo','Acronimo'])

In [75]:
df_teams_copy[7].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28 entries, 0 to 27
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Equipo                   28 non-null     object 
 1   Cantidad_agentes_libres  28 non-null     int64  
 2   Valor_contrato_total     28 non-null     int64  
 3   Acronimo                 28 non-null     object 
 4   Victorias                28 non-null     int64  
 5   Juegos totales           28 non-null     int64  
 6   Playoffs                 28 non-null     int64  
 7   Pennants won             28 non-null     int64  
 8   WS ganadas               28 non-null     int64  
 9   Promedio_victorias       28 non-null     float64
 10  Estado                   28 non-null     object 
 11  Cantidad de equipos      28 non-null     int64  
dtypes: float64(1), int64(8), object(3)
memory usage: 2.8+ KB


In [76]:
df_salary_copy[7].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1036 entries, 0 to 1035
Data columns (total 23 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Jugador                  1036 non-null   object 
 1   Anio                     1036 non-null   object 
 2   Posicion                 1036 non-null   object 
 3   Acronimo                 1036 non-null   object 
 4   Sueldo_base              1036 non-null   int64  
 5   Bono_por_firma           1036 non-null   int64  
 6   Sueldo_regular           1036 non-null   int64  
 7   Sueldo_ajustado          1036 non-null   int64  
 8   Sueldo_porcentual        1036 non-null   float64
 9   Pago_efectivo            1036 non-null   int64  
 10  Valor_contrato_promedio  1036 non-null   int64  
 11  Anios_de_contrato        1036 non-null   int64  
 12  Valor_del_contrato       1036 non-null   int64  
 13  Ganancias                1036 non-null   int64  
 14  Anio_de_agente_libre    

Ahora, unamos las bases de datos sobre los equipos a las bases de datos de los salarios

In [77]:
for year in range(0,period):
    df_salary_copy[year] = pd.merge(df_teams_copy[year], df_salary_copy[year], on = 'Acronimo')

In [78]:
df_salary_copy[0].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40 entries, 0 to 39
Data columns (total 34 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Equipo                   40 non-null     object 
 1   Cantidad_agentes_libres  40 non-null     int64  
 2   Valor_contrato_total     40 non-null     int64  
 3   Acronimo                 40 non-null     object 
 4   Victorias                40 non-null     int64  
 5   Juegos totales           40 non-null     int64  
 6   Playoffs                 40 non-null     int64  
 7   Pennants won             40 non-null     int64  
 8   WS ganadas               40 non-null     int64  
 9   Promedio_victorias       40 non-null     float64
 10  Estado                   40 non-null     object 
 11  Cantidad de equipos      40 non-null     int64  
 12  Jugador                  40 non-null     object 
 13  Anio                     40 non-null     object 
 14  Posicion                 40 

Debido a que la mayoría de los jugadores juega tanto en la ofensiva como la defensiva es que tenemos que borrar los duplicados de la columna de la posición.

In [79]:
for year in range(0,period):
    df_hitting_copy[year] = pd.merge(df_hitting_copy[year], df_salary_copy[year], on = 'Jugador')
    df_pitching_copy[year] = pd.merge(df_pitching_copy[year], df_salary_copy[year], on = 'Jugador')

In [80]:
for year in range(0,period):
    df_pitching_copy[year]['Porcentaje_juegos'] = df_pitching_copy[year]['Juegos']/df_pitching_copy[year]['Juegos totales']

In [81]:
df_hitting_copy[3].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 668 entries, 0 to 667
Data columns (total 64 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Jugador                             668 non-null    object 
 1   Juegos                              668 non-null    int64  
 2   Porcentaje_juegos                   668 non-null    float64
 3   Juegos_iniciados                    668 non-null    int64  
 4   Porcentaje_juegos_iniciados         668 non-null    float64
 5   At-bats                             668 non-null    int64  
 6   Bateos                              668 non-null    int64  
 7   Dobles                              668 non-null    int64  
 8   Triples                             668 non-null    int64  
 9   Home-runs                           668 non-null    int64  
 10  Runs-batted-in                      668 non-null    int64  
 11  Bateos_promedio                     668 non-n

In [82]:
df_pitching_copy[3].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 498 entries, 0 to 497
Data columns (total 69 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Jugador                  498 non-null    object 
 1   Juegos                   498 non-null    int64  
 2   Juegos_iniciados         498 non-null    int64  
 3   Inning_pitched           498 non-null    float64
 4   Bateos                   498 non-null    int64  
 5   Carreras                 498 non-null    int64  
 6   Carreras_ganadas         498 non-null    int64  
 7   Walks                    498 non-null    int64  
 8   Strike-outs              498 non-null    int64  
 9   Wins                     498 non-null    int64  
 10  Losses                   498 non-null    int64  
 11  Saves                    498 non-null    int64  
 12  WHIP                     498 non-null    float64
 13  ERA                      498 non-null    float64
 14  WAR                      4

Para facilitar la observación de las trnasformaciones de manera más eficiente, ordenemos alfabéticamente la base de datos de acuerdo al nombre de las columnas.

In [83]:
for year in range(0,period):
    # Ordenando alfabéticamente
    df_salary_copy[year].sort_index(axis = 1, inplace = True)
    df_hitting_copy[year].sort_index(axis = 1, inplace = True)
    df_pitching_copy[year].sort_index(axis = 1, inplace = True)
    df_free_agents_copy[year].sort_index(axis = 1, inplace = True)
    
    # Reiniciando los índices
    df_salary_copy[year].reset_index(drop = True, inplace = True)
    df_hitting_copy[year].reset_index(drop = True, inplace = True)
    df_pitching_copy[year].reset_index(drop = True, inplace = True)
    df_free_agents_copy[year].reset_index(drop = True, inplace = True)

## Variables del periodo t-1

Lo que haremos será un *merge* de las bases de datos del año $t$ con el año $t-1$ sobre los jugadores. La razón de esto es que solo nos interesan los jugadores que han sido agentes libres por más de un año.

Si la primera base de datos es del año 2011, entonces tendremos que empezar en el año 2012. Creemos los dataframes que contendrán los datos para el modelo. Para que no se sobrepongan todos los periodos, crearemos dataframes auxiliares para guardar los nuevos datos

In [84]:
hitting_merge = ['Juegos_iniciados', 'Porcentaje_juegos_iniciados', 'At-bats', 'Bateos',
                 'Dobles', 'Triples', 'Home-runs', 'Runs-batted-in', 'Bateos_promedio',
                 'Porcentaje_on-base', 'Porcentaje_slugging', 'TVS',
                 'Porcentaje_On-base-plus-slugging', 'WAR',
                 'Juegos_iniciados_2', 'Porcentaje_juegos_iniciados_2', 'At-bats_2', 'Bateos_2',
                 'Dobles_2', 'Triples_2', 'Home-runs_2', 'Runs-batted-in_2', 'Bateos_promedio_2',
                 'Porcentaje_on-base_2', 'Porcentaje_slugging_2', 'TVS_2',
                 'Porcentaje_On-base-plus-slugging_2', 'WAR_2']	
pitching_merge = ['Inning_pitched', 'Bateos_en_contra', 'Carreras_en_contra',
                  'Carreras_ganadas', 'Walks', 'Strike-outs', 'Wins', 'Losses',
                  'Saves', 'WHIP', 'ERA', 'WAR', 'TVS', 'Dominio', 'Control',
                  'Comando',
                  'Inning_pitched_2', 'Bateos_2', 'Carreras_2',
                  'Carreras_ganadas_2', 'Walks_2', 'Strike-outs_2', 'Wins_2', 'Losses_2',
                  'Saves_2', 'WHIP_2', 'ERA_2', 'WAR_2', 'TVS_2', 'Dominio_2', 'Control_2',
                  'Comando_2']

In [85]:
df_hitters_copy = [None]*period
df_pitchers_copy = [None]*period

In [86]:
for year in range(0,period):
    df_hitters_copy[year] = df_hitting_copy[year].copy()
    df_pitchers_copy[year] = df_pitching_copy[year].copy()

In [87]:
for year in range(1,period):    
    df_hitting_copy[year] = pd.merge(df_hitters_copy[year], df_hitters_copy[year-1], on = 'Jugador')
    df_pitching_copy[year] = pd.merge(df_pitchers_copy[year], df_pitchers_copy[year-1], on = 'Jugador')

A continuación se verifica que la cantidad de columnas sea la misma, salvo por el primer periodo

In [88]:
"""for name in df_pitching_copy[11].columns:
    print(name)"""

'for name in df_pitching_copy[11].columns:\n    print(name)'

In [89]:
for year in range(0,period):
    print(df_hitting_copy[year].columns.shape)
    
for year in range(0,period):    
    print(df_pitching_copy[year].columns.shape)

(64,)
(127,)
(127,)
(127,)
(127,)
(127,)
(127,)
(127,)
(127,)
(127,)
(127,)
(127,)
(69,)
(137,)
(137,)
(137,)
(137,)
(137,)
(137,)
(137,)
(137,)
(137,)
(137,)
(137,)


In [90]:
for year in range(1,period):       
    df_pitching_copy[year].columns = df_pitching_copy[year].columns.str.replace('_x', '_t')
    df_pitching_copy[year].columns = df_pitching_copy[year].columns.str.replace('_y', '_t_1')
    df_pitching_copy[year].columns = df_pitching_copy[year].columns.str.replace('-', '_')
    df_pitching_copy[year].columns = df_pitching_copy[year].columns.str.replace(' ', '_')
    df_pitching_copy[year].drop(['ln_Sueldo_base_t_1', 'ln_Sueldo_ajustado_t_1', 'ln_Sueldo_regular_t_1'],
                           axis = 1, inplace = True)
    df_pitching_copy[year] = df_pitching_copy[year].sort_values(by = 'Jugador', ascending = True)
    df_pitching_copy[year].reset_index(drop = True, inplace = True)
    
    df_hitting_copy[year].columns = df_hitting_copy[year].columns.str.replace('_x', '_t')
    df_hitting_copy[year].columns = df_hitting_copy[year].columns.str.replace('_y', '_t_1')
    df_hitting_copy[year].columns = df_hitting_copy[year].columns.str.replace('-', '_')
    df_hitting_copy[year].columns = df_hitting_copy[year].columns.str.replace(' ', '_')
    df_hitting_copy[year].drop(['ln_Sueldo_base_t_1', 'ln_Sueldo_ajustado_t_1', 'ln_Sueldo_regular_t_1'],
                          axis = 1, inplace = True)
    df_hitting_copy[year] = df_hitting_copy[year].sort_values(by = 'Jugador', ascending = True)
    df_hitting_copy[year].reset_index(drop = True, inplace = True)
    
    # Reordenando las columnas
    df_hitting_copy[year].sort_index(axis = 1, inplace = True)
    df_pitching_copy[year].sort_index(axis = 1, inplace = True)

In [91]:
for name in df_pitching_copy[11].columns:
    print(name)

Acronimo_t
Acronimo_t_1
Altura_t
Altura_t_1
Anio_de_agente_libre_t
Anio_de_agente_libre_t_1
Anio_t
Anio_t_1
Anios_de_contrato_t
Anios_de_contrato_t_1
Antiguedad_t
Antiguedad_t_1
Bateos_2_t
Bateos_2_t_1
Bateos_t
Bateos_t_1
Bono_por_firma_t
Bono_por_firma_t_1
Cantidad_agentes_libres_t
Cantidad_agentes_libres_t_1
Cantidad_de_equipos_t
Cantidad_de_equipos_t_1
Carreras_2_t
Carreras_2_t_1
Carreras_ganadas_2_t
Carreras_ganadas_2_t_1
Carreras_ganadas_t
Carreras_ganadas_t_1
Carreras_t
Carreras_t_1
Comando_2_t
Comando_2_t_1
Comando_t
Comando_t_1
Control_2_t
Control_2_t_1
Control_t
Control_t_1
Dominio_2_t
Dominio_2_t_1
Dominio_t
Dominio_t_1
ERA_2_t
ERA_2_t_1
ERA_t
ERA_t_1
Edad_al_firmar_t
Edad_al_firmar_t_1
Edad_t
Edad_t_1
Equipo_t
Equipo_t_1
Estado_t
Estado_t_1
Ganancias_t
Ganancias_t_1
Inning_pitched_2_t
Inning_pitched_2_t_1
Inning_pitched_t
Inning_pitched_t_1
Juegos_iniciados_t
Juegos_iniciados_t_1
Juegos_t
Juegos_t_1
Juegos_totales_t
Juegos_totales_t_1
Jugador
Losses_2_t
Losses_2_t_1
Losses_t

Debido a que muchas de las variables del periodo $t_1$ pueden funcionar como controles más realistas, se optarán por dejarlas a excepción de la columna que contiene el dato del año al que hace referencia el dataframe del periodo $t_1$, es decir, la columna *Anio_t_1*. Esto se hará para *pitchers* y *hitters*. Por razones análogas, también de omitirá la columna que indica la cantidad de equipos en determinado estado ya que en el periodo de análisis es invariante.

Para facilitar la escritura del código, entenderemos la columna *Anio* como la columna *Anio_t*.

In [92]:
for year in range(1,period):
    df_pitching_copy[year].drop(['Anio_t_1', 'Estado_t_1', 'Edad_t_1'],
                           axis = 1, inplace = True)
    
    df_hitting_copy[year].drop(['Anio_t_1', 'Estado_t_1', 'Edad_t_1'],
                           axis = 1, inplace = True)
    
    # Reordenando las columnas
    df_hitting_copy[year].sort_index(axis = 1, inplace = True)
    df_pitching_copy[year].sort_index(axis = 1, inplace = True)
    
    # Reiniciando índice
    df_hitting_copy[year].reset_index(drop = True, inplace = True)
    df_pitching_copy[year].reset_index(drop = True, inplace = True)

Cambiemos el súfijo de las basses de datos del año del 2011

In [93]:
year = 0
# Reiniciando los índices
df_hitting_copy[year] = df_hitting_copy[year].add_suffix('_t')
df_pitching_copy[year] = df_pitching_copy[year].add_suffix('_t')
# Corrección de columna del jugador
df_hitting_copy[year].columns = df_hitting_copy[year].columns.str.replace('Jugador_t', 'Jugador')
df_pitching_copy[year].columns = df_pitching_copy[year].columns.str.replace('Jugador_t', 'Jugador')

In [94]:
print("Salarios")
print(df_salary_copy[year].columns)
print("\n")
print("Hitters")
print(df_hitting_copy[year].columns)
print("\n")
print("Pitchers")
print(df_pitching_copy[year].columns)
print("\n")
print("Free agents")
print(df_free_agents_copy[year].columns)
print("\n")

Salarios
Index(['Acronimo', 'Altura', 'Anio', 'Anio_de_agente_libre',
       'Anios_de_contrato', 'Antiguedad', 'Bono_por_firma',
       'Cantidad de equipos', 'Cantidad_agentes_libres', 'Edad',
       'Edad_al_firmar', 'Equipo', 'Estado', 'Ganancias', 'Juegos totales',
       'Jugador', 'Pago_efectivo', 'Pennants won', 'Peso', 'Playoffs',
       'Posicion', 'Promedio_victorias', 'Sueldo_ajustado', 'Sueldo_base',
       'Sueldo_porcentual', 'Sueldo_regular', 'Valor_contrato_promedio',
       'Valor_contrato_total', 'Valor_del_contrato', 'Victorias', 'WS ganadas',
       'ln_Sueldo_ajustado', 'ln_Sueldo_base', 'ln_Sueldo_regular'],
      dtype='object')


Hitters
Index(['Acronimo_t', 'Altura_t', 'Anio_t', 'Anio_de_agente_libre_t',
       'Anios_de_contrato_t', 'Antiguedad_t', 'At-bats_t', 'At-bats_2_t',
       'Bateos_t', 'Bateos_2_t', 'Bateos_promedio_t', 'Bateos_promedio_2_t',
       'Bono_por_firma_t', 'Cantidad de equipos_t',
       'Cantidad_agentes_libres_t', 'Dobles_t', 'Dobles_2

## Segmentación por Agentes libres

Separaremos los pitchers y hitters en dos grupos:

- Agentes libres.
- No agentes libres.

In [95]:
for year in range(0,period):
    # Filtrando los agentes libres
    df_hitters_free_agents[year] = pd.merge(df_free_agents_copy[year],
                                            df_hitting_copy[year], on = 'Jugador')
    df_pitchers_free_agents[year] = pd.merge(df_free_agents_copy[year],
                                             df_pitching_copy[year], on = 'Jugador')
    # FIltrando los que no son agentes libres
    df_hitters_no_free_agents[year] = df_hitting_copy[year][~df_hitting_copy[year].Jugador.isin(df_hitters_free_agents[year].Jugador)]
    df_pitchers_no_free_agents[year] = df_pitching_copy[year][~df_pitching_copy[year].Jugador.isin(df_pitchers_free_agents[year].Jugador)]
    
    # Reiniciando el índice
    df_hitters_free_agents[year] = df_hitters_free_agents[year].reindex(sorted(df_hitters_free_agents[year].columns), axis=1)
    df_pitchers_free_agents[year] = df_pitchers_free_agents[year].reindex(sorted(df_pitchers_free_agents[year].columns), axis=1)
    df_hitters_no_free_agents[year] = df_hitters_no_free_agents[year].reindex(sorted(df_hitters_no_free_agents[year].columns), axis=1)
    df_pitchers_no_free_agents[year] = df_pitchers_no_free_agents[year].reindex(sorted(df_pitchers_no_free_agents[year].columns), axis=1)    

Veamos los contenidos de las nuevas bases de datos

In [96]:
print("FA - Hitters:")
df_hitters_free_agents[9].info()
print("\n FA - Pitchers:")
df_pitchers_free_agents[9].info()
print("\n No FA - Hitters:")
df_hitters_no_free_agents[9].info()
print("\n No FA - Hitters:")
df_pitchers_no_free_agents[9].info()

FA - Hitters:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 56 entries, 0 to 55
Columns: 126 entries, Acronimo_t to ln_Sueldo_regular_t
dtypes: float64(40), int64(75), object(11)
memory usage: 55.6+ KB

 FA - Pitchers:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 75 entries, 0 to 74
Columns: 136 entries, Acronimo_t to ln_Sueldo_regular_t
dtypes: float64(44), int64(81), object(11)
memory usage: 80.3+ KB

 No FA - Hitters:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 225 entries, 1 to 280
Columns: 121 entries, Acronimo_t to ln_Sueldo_regular_t
dtypes: float64(40), int64(72), object(9)
memory usage: 214.5+ KB

 No FA - Hitters:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 282 entries, 0 to 352
Columns: 131 entries, Acronimo_t to ln_Sueldo_regular_t
dtypes: float64(44), int64(78), object(9)
memory usage: 290.8+ KB


In [97]:
print("FA - Hitters:")
for year in range(0,period):
    print(df_hitters_free_agents[year].shape)
print("\n FA - Pitchers:")
for year in range(0,period):
    print(df_pitchers_free_agents[year].shape)

FA - Hitters:
(0, 69)
(0, 126)
(70, 126)
(91, 126)
(97, 126)
(117, 126)
(93, 126)
(80, 126)
(85, 126)
(56, 126)
(49, 126)
(49, 126)

 FA - Pitchers:
(0, 74)
(1, 136)
(47, 136)
(70, 136)
(86, 136)
(89, 136)
(68, 136)
(73, 136)
(74, 136)
(75, 136)
(75, 136)
(93, 136)


Por último, para facilitar futuras aplicaciones, pasemos todos los nombres de las columnas a miníscula

In [98]:
for year in range(0,period):
    df_hitters_free_agents[year].rename(columns = str.lower)
    df_pitchers_free_agents[year].rename(columns = str.lower)
    df_hitters_no_free_agents[year].rename(columns = str.lower)
    df_pitchers_no_free_agents[year].rename(columns = str.lower)

### Etiquetas para los agentes libres

Crearemos un etiqueta para indicar si el pitcher o hitter es  un agente libre o no.

## Panel Data

Con el objetivo de contar con una base de datos en estructura panel, uniremos las bases de datos

In [99]:
# Inicialización del panel
df_panel_all_hitter = df_hitting_copy[0]
df_panel_all_pitcher = df_pitching_copy[0]

for year in range(1,period):
    # Hitter
    df_panel_all_hitter = pd.concat([df_panel_all_hitter, df_hitting_copy[year]])
    
    # Pitcher
    df_panel_all_pitcher = pd.concat([df_panel_all_pitcher, df_pitching_copy[year]])

Veamos las estadísticas descriptivas de los panel

In [100]:
df_panel_all_hitter[['ln_Sueldo_ajustado_t']].describe()

,ln_Sueldo_ajustado_t
count,4243.000000
mean,14.212871
std,1.591619
min,8.620832
25%,13.122363
50%,14.000914
75%,15.607270
max,17.440000


In [101]:
df_panel_all_pitcher.describe()

,Altura_t,Anio_de_agente_libre_t,Anios_de_contrato_t,Antiguedad_t,Bateos_t,Bateos_2_t,Bono_por_firma_t,Cantidad de equipos_t,Cantidad_agentes_libres_t,Carreras_t,...,WAR_2_t_1,WAR_t_1,WHIP_2_t_1,WHIP_t_1,WS_ganadas_t,WS_ganadas_t_1,Walks_2_t_1,Walks_t_1,Wins_2_t_1,Wins_t_1
count,3677.000000,3677.000000,3677.000000,3677.000000,3677.000000,3677.000000,3.677000e+03,10.0,3677.000000,3677.000000,...,3667.000000,3667.000000,3667.000000,3667.000000,3667.000000,3667.000000,3667.000000,3667.000000,3667.000000,3667.000000
mean,6.262453,1367.335328,1.607017,0.799837,72.470220,8475.991569,1.050010e+05,1.0,5.482459,36.696492,...,3.096443,1.327190,2.023568,1.328459,3.963185,4.020725,998.272430,25.387783,39.492228,4.505590
std,0.208318,945.957929,1.341067,1.043749,56.788518,11645.808941,6.036374e+05,0.0,3.039821,28.437345,...,6.611205,1.155583,7.520555,0.508758,5.409475,5.539927,1364.815296,18.810353,67.355285,4.381452
min,5.567014,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000e+00,1.0,1.000000,0.000000,...,0.000100,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,6.100000,0.000000,1.000000,0.000000,31.000000,961.000000,0.000000e+00,1.0,3.000000,16.000000,...,0.336400,0.580000,1.299600,1.140000,1.000000,1.000000,121.000000,11.000000,1.000000,1.000000
50%,6.300000,2019.000000,1.000000,1.000000,53.000000,2809.000000,0.000000e+00,1.0,5.000000,27.000000,...,1.416100,1.190000,1.638400,1.280000,2.000000,2.000000,400.000000,20.000000,9.000000,3.000000
75%,6.400000,2023.000000,1.000000,1.000000,112.000000,12544.000000,0.000000e+00,1.0,7.000000,56.000000,...,2.160900,1.470000,2.073600,1.440000,5.000000,5.000000,1369.000000,37.000000,49.000000,7.000000
max,6.900000,2029.000000,9.000000,8.000000,314.000000,98596.000000,1.000000e+07,1.0,18.000000,162.000000,...,93.896100,9.690000,441.000000,21.000000,27.000000,27.000000,11236.000000,106.000000,484.000000,22.000000


In [102]:
df_panel_all_hitter.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4243 entries, 0 to 296
Columns: 135 entries, Acronimo_t to WS_ganadas_t_1
dtypes: float64(100), int64(26), object(9)
memory usage: 4.4+ MB


In [103]:
df_panel_all_pitcher.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3677 entries, 0 to 400
Columns: 137 entries, Acronimo_t to Wins_t_1
dtypes: float64(95), int64(33), object(9)
memory usage: 3.9+ MB


Verifquemos que no haya problemas con valores *NaN* o *infinitos*

Valores *NaN*:

In [104]:
"""for name in df_panel_all_hitter.columns:
    print(name)
    if type(name) != str:
        for element in range(0,len(df_panel_all_hitter[name])):
            if pd.isna(df_panel_all_hitter[name].iloc[element]) == True:
                print(str(element) +  '  ' + str(df_panel_all_hitter[name].iloc[element]))"""

"for name in df_panel_all_hitter.columns:\n    print(name)\n    if type(name) != str:\n        for element in range(0,len(df_panel_all_hitter[name])):\n            if pd.isna(df_panel_all_hitter[name].iloc[element]) == True:\n                print(str(element) +  '  ' + str(df_panel_all_hitter[name].iloc[element]))"

In [105]:
"""for name in df_panel_all_pitcher.columns:
    print(name)
    if type(name) != str:
        for element in range(0,len(df_panel_all_pitcher[name])):
            if pd.isna(df_panel_all_pitcher[name].iloc[element]) == True:
                print(str(element) +  '  ' + str(df_panel_all_pitcher[name].iloc[element]))"""

"for name in df_panel_all_pitcher.columns:\n    print(name)\n    if type(name) != str:\n        for element in range(0,len(df_panel_all_pitcher[name])):\n            if pd.isna(df_panel_all_pitcher[name].iloc[element]) == True:\n                print(str(element) +  '  ' + str(df_panel_all_pitcher[name].iloc[element]))"

Valores *infinitos*

In [106]:
for name in df_panel_all_hitter.columns:
    print(name)
    if type(name) != str:
        for element in range(0,len(df_panel_all_hitter[name])):
            if math.isinf(df_panel_all_hitter[name].iloc[element]) == True:
                print(str(element) +  '  ' + str(df_panel_all_hitter[name].iloc[element]))

Acronimo_t
Altura_t
Anio_t
Anio_de_agente_libre_t
Anios_de_contrato_t
Antiguedad_t
At-bats_t
At-bats_2_t
Bateos_t
Bateos_2_t
Bateos_promedio_t
Bateos_promedio_2_t
Bono_por_firma_t
Cantidad de equipos_t
Cantidad_agentes_libres_t
Dobles_t
Dobles_2_t
Edad_t
Edad_al_firmar_t
Equipo_t
Estado_t
Ganancias_t
Home-runs_t
Home-runs_2_t
Juegos_t
Juegos totales_t
Juegos_iniciados_t
Juegos_iniciados_2_t
Jugador
Pago_efectivo_t
Pennants won_t
Peso_t
Playoffs_t
Porcentaje_On-base-plus-slugging_t
Porcentaje_On-base-plus-slugging_2_t
Porcentaje_juegos_t
Porcentaje_juegos_iniciados_t
Porcentaje_juegos_iniciados_2_t
Porcentaje_on-base_t
Porcentaje_on-base_2_t
Porcentaje_slugging_t
Porcentaje_slugging_2_t
Posicion_t
Promedio_victorias_t
Runs-batted-in_t
Runs-batted-in_2_t
Sueldo_ajustado_t
Sueldo_base_t
Sueldo_porcentual_t
Sueldo_regular_t
TVS_t
TVS_2_t
Triples_t
Triples_2_t
Valor_contrato_promedio_t
Valor_contrato_total_t
Valor_del_contrato_t
Victorias_t
WAR_t
WAR_2_t
WS ganadas_t
ln_Sueldo_ajustado_t
ln

In [107]:
for name in df_panel_all_pitcher.columns:
    print(name)
    if type(name) != str:
        for element in range(0,len(df_panel_all_pitcher[name])):
            if math.isinf(df_panel_all_pitcher[name].iloc[element]) == True:
                print(str(element) +  '  ' + str(df_panel_all_pitcher[name].iloc[element]))

Acronimo_t
Altura_t
Anio_t
Anio_de_agente_libre_t
Anios_de_contrato_t
Antiguedad_t
Bateos_t
Bateos_2_t
Bono_por_firma_t
Cantidad de equipos_t
Cantidad_agentes_libres_t
Carreras_t
Carreras_2_t
Carreras_ganadas_t
Carreras_ganadas_2_t
Comando_t
Comando_2_t
Control_t
Control_2_t
Dominio_t
Dominio_2_t
ERA_t
ERA_2_t
Edad_t
Edad_al_firmar_t
Equipo_t
Estado_t
Ganancias_t
Inning_pitched_t
Inning_pitched_2_t
Juegos_t
Juegos totales_t
Juegos_iniciados_t
Jugador
Losses_t
Losses_2_t
Pago_efectivo_t
Pennants won_t
Peso_t
Playoffs_t
Porcentaje_juegos_t
Posicion_t
Promedio_victorias_t
Saves_t
Saves_2_t
Strike-outs_t
Strike-outs_2_t
Sueldo_ajustado_t
Sueldo_base_t
Sueldo_porcentual_t
Sueldo_regular_t
TVS_t
TVS_2_t
Valor_contrato_promedio_t
Valor_contrato_total_t
Valor_del_contrato_t
Victorias_t
WAR_t
WAR_2_t
WHIP_t
WHIP_2_t
WS ganadas_t
Walks_t
Walks_2_t
Wins_t
Wins_2_t
ln_Sueldo_ajustado_t
ln_Sueldo_base_t
ln_Sueldo_regular_t
Acronimo_t_1
Altura_t_1
Anio_de_agente_libre_t_1
Anios_de_contrato_t_1
Antig

In [108]:
df_panel_all_hitter.sort_index(axis = 1, inplace = True)
df_panel_all_pitcher.sort_index(axis = 1, inplace = True)

Exportemos los paneles

In [128]:
df_panel_all_hitter.to_csv('ETL_Data/Cumulative/Panel/Hitter/visual_panel' + '.csv',
                           index = False)
df_panel_all_pitcher.to_csv('ETL_Data/Cumulative/Panel/Pitcher/visual_panel' + '.csv',
                            index = False)